In [4]:
from postgres import PostGresClient
import pandas as pd
import pandas.io.sql as psql
import time

In [5]:
#loading in data
df = PostGresClient().load_table('flavor_freq')
    

c:\Anaconda\envs\wine_recommend\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [6]:
df.head()
print(df.shape[0])

1663243


In [15]:
tmp = df[['wine_id', 'keyword_count']].groupby('wine_id').sum('keyword_count')
tmp[tmp.index == 51644]

,keyword_count
wine_id,
51644,207


In [35]:
#create a term freq matrix 

def create_tf(df):

    strt = time.time()
    #create the df where rows are wine_id and columns are keywords
    words = df.keyword_name.unique()
    wines = df.wine_id.unique()

    num_words_df = df[['wine_id', 'keyword_count']].groupby('wine_id').sum('keyword_count')

    tf = pd.DataFrame(0, columns=words, index = wines)
    for i in range(df.shape[0]):
        #finding index of wine_id  getting keyword and count
        idx_num = tf.index.get_loc(df.wine_id[i])
        kywrd = df.keyword_name[i]
        cnt = df.keyword_count[i]
        num_wrds = num_words_df[num_words_df.index == num_words_df.index.get_loc(df.wine_id[i])]

        #updating record in table
        tf[kywrd].iloc[idx_num] = cnt / num_wrds

    end = time.time()
    print(f"----- Processing took {end - strt} seconds to complete -----")
    return tf





----- Processing took 341.79496335983276 seconds to complete -----


In [37]:
tf = create_tf(df)
tf.head()

,oak,tobacco,vanilla,chocolate,coffee,clove,butter,nutmeg,cigar,dark chocolate,...,incense smoke,dried chili pepper,mushroom broth,wood ear,dried red chili,gingersnap,chayote squash,peppercress,lavage,true cinnamon
51644,35,18,12,5,3,2,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1128181,16,5,7,2,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6417379,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73249,48,9,24,5,4,5,4,2,2,2,...,0,0,0,0,0,0,0,0,0,0
1128885,8,0,7,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#create idf matrix

def create_idf(df):
    

In [38]:
def create_keywords_col(df):
    df['primary_full_keywords'] = df[['wine_id','primary_keywords']].groupby(['wine_id',])['primary_keywords'].transform(lambda x: '|'.join(x))
    df['secondary_full_keywords'] = df[['wine_id','secondary_keywords']].groupby(['wine_id',])['secondary_keywords'].transform(lambda x: '|'.join(x))

    #drop duplicate rows
    df = df.drop_duplicates(subset=['wine_id'])
    for i in range(2):#range(df.shape[0]):
        keywords = df['primary_full_keywords'][i].split('|')
        for j in keywords:
            print(j)
    

In [39]:
df

,wine_id,group,keyword_id,keyword_name,keyword_count
0,51644,oak,292,oak,35
1,51644,oak,422,tobacco,18
2,51644,oak,434,vanilla,12
3,51644,oak,101,chocolate,5
4,51644,oak,117,coffee,3
...,...,...,...,...,...
1663238,1589325,oak,422,tobacco,1
1663239,1589325,oak,70,butter,1
1663240,1589325,oak,434,vanilla,1
1663241,1589325,floral,436,violet,1


In [40]:
create_keywords_col(df)

KeyError: "['primary_keywords'] not in index"